In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

C:\Users\mook\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [4]:
sample = (" Authorities in Turkey -- who say Khashoggi"
          "was murdered by a hit squad sent from Riyadh"
          "-- are still searching for the journalist's remains."
          " Earlier this week, the chief prosecutor's office "
          "said Khashoggi's body was dismembered after he was str"
          "angled, while the Washington Post reported investigators"
          "were looking into the theory that the body was dissolved "
          "in acid. A source close to the Saudi Royal Palace has de"
          "nied any knowledge of the body's whereabouts.")
idx2char = list(set(sample))
char2idx = {c:i for i,c in enumerate(idx2char)}

In [32]:
input_dim = len(idx2char)
hidden_size = len(idx2char)
num_classes = len(idx2char)
sequence_length = 10
learning_rate = 0.1

In [33]:
dataX = []
dataY = []
for i in range(0, len(sample)-sequence_length):
    x_str = sample[i:i+sequence_length]
    y_str = sample[i+1:i+sequence_length+1]
    x = [char2idx[c] for c in x_str]
    y = [char2idx[c] for c in y_str]
    dataX.append(x)
    dataY.append(y)

In [ ]:
batch_size = len(dataX)

sample_idx = [char2idx[c] for c in sample]
x_data = [sample_idx[:-1]]
y_data = [sample_idx[1:]]

In [ ]:
X = tf.placeholder(tf.int32, shape=[None,sequence_length])
Y = tf.placeholder(tf.int32, shape=[None,sequence_length])
x_one_hot = tf.one_hot(X, input_dim)


def lstm_cell():
    return tf.contrib.rnn.BasicLSTMCell(hidden_size, state_is_tuple=True)
multi_cells = tf.contrib.rnn.MultiRNNCell([lstm_cell()]*3, state_is_tuple=True)
initial_state = cell.zero_state(batch_size, tf.float32)
outputs, _states = tf.nn.dynamic_rnn(multi_cells, X_one_hot, dtype=tf.float32, initial_state=initial_state)

X_for_fc = tf.reshape(outputs, [-1,hidden_size])
outputs = tf.contrib.layers.fully_connected(X_for_fc, num_classes, activation_fn=None)
outputs = tf.reshape(outputs, [batch_size,sequence_length, num_classes])
weights = tf.ones([batch_size, sequence_length])
sequence_loss = tf.contrib.seq2seq.sequence_loss(logits=outputs, targets=Y,
                                                weights=weights)
mean_loss = tf.reduce_mean(sequence_loss)
train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(mean_loss)

prediction = tf.argmax(outputs, axis=2)


In [10]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
for step in range(4000):
    l,_ = sess.run([loss,train], feed_dict={X:x_data, Y:y_data})
    result = sess.run(prediction, feed_dict={X:x_one_hot})
    if step % 100 == 0:
        print ("Step:",step,"Loss:",l,"Prediction:", result)
        result_str = [idx2char[c] for c in np.squeeze(result)]
        print ("Prediction:","".join(result_str))

ValueError: Cannot feed value of shape (1, 32) for Tensor 'Placeholder:0', which has shape '(?, 32, 16)'

In [ ]:
result = sess.run(outputs, feed_dict={X:dataX})
for j,result in enumerate(result):
    index = np.argmax(result, axis=1)
    if j is 0:
        print("".join([char2idx[t] for t in index]), end='')
    else:
        print(idx2char[index[-1]], end='')

